In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter


# Parameters
Nx, Ny = 100, 100
dx, dy = 1.0, 1.0
dt = 0.01
M = 1.0
kappa = 0.02
Nt = 200

# Boundary condition
boundary_condition = 'Neumann'

# Initialize
# Initialize variables and arrays
c_hat = np.empty((N, N), dtype=np.complex64)  # Fourier transformed concentration
dfdc_hat = np.empty((N, N), dtype=np.complex64)  # Fourier transformed derivative of free energy density
c = np.empty((Nsteps, N, N), dtype=np.float32)  # Array to store concentration at each time step
# Initialize wavenumbers and squared wavenumber
"""
 generate kx and ky are arrays of angular frequencies along the x and y axes, respectively 
 for a given domain size N and grid dx
 np.fft.fftfreq(N, d=dx) returns an array of frequencies [f_0, f_1, ..., f_{N-1}] 
"""
kx = ky = np.fft.fftfreq(N, d=dx) * 2 * np.pi
"""
 K is a framework that allows us to apply mathematical operations over the entire Fourier-transformed space simultaneously
"""

K = np.array(np.meshgrid(kx, ky, indexing='ij'), dtype=np.float32) 
"""
K2 squared wavenumber
"""
K2 = np.sum(K * K, axis=0, dtype=np.float32)


# Set up anti-aliasing
"""
solve partial differential equations (PDEs) 
by transforming them into the frequency domain, aliasing can be particularly problematic. 
"""
kmax_dealias = kx.max() * 2.0 / 3.0
dealias = np.array((np.abs(K[0]) < kmax_dealias) * (np.abs(K[1]) < kmax_dealias), dtype=bool)



rng = np.random.default_rng(12345)
noise = 0.1
c0 = 0.5
c = c0 + noise * rng.standard_normal((Nx, Ny))


"""
 The bulk free energy density f(c) = Wc^2(1-c)^2
"""
def fbulk(c):
    return W*c**2*(1-c)*c**2

"""
 The derivative of bulk free energy density f(c) = Wc^2(1-c)^2
"""
def dfdc(c):
    return 2*W*(c*(1-c)**2-(1-c)*c**2)
def calculate_mu(c):
    return -c**3 + c - 4  # For simplicity; this will depend on your specific problem

def rk4_step(c, dt):
    # Implement RK4 logic here
    return updated_mu

def adams_moulton_step(c, mu, dt):
    # Implement Adams-Moulton logic here
    return updated_c
# Function to calculate Laplacian
def laplacian(p):
    lap = np.roll(p, 1, axis=0) + np.roll(p, -1, axis=0) + np.roll(p, 1, axis=1) + np.roll(p, -1, axis=1) - 4 * p
    lap /= dx**2
    return lap

# Runge-Kutta for first 3 steps
for n in range(3):
    k1 = laplacian(phi**3 - phi - laplacian(phi))
    k2 = laplacian((phi + 0.5 * dt * k1)**3 - (phi + 0.5 * dt * k1) - laplacian(phi + 0.5 * dt * k1))
    k3 = laplacian((phi + 0.5 * dt * k2)**3 - (phi + 0.5 * dt * k2) - laplacian(phi + 0.5 * dt * k2))
    k4 = laplacian((phi + dt * k3)**3 - (phi + dt * k3) - laplacian(phi + dt * k3))
    
    phi += dt / 6.0 * (k1 + 2 * k2 + 2 * k3 + k4)

# Adams-Moulton for subsequent steps
for n in range(3, nt):
    # Implicit scheme, nonlinear equation solved using fsolve
    for i in range(1, nx-1):
        for j in range(1, ny-1):
            func = lambda new_phi_ij : new_phi_ij - phi[i,j] - dt * laplacian(new_phi_ij**3 - new_phi_ij - laplacian(new_phi_ij))
            new_phi[i, j] = fsolve(func, phi[i, j])[0]
    phi = np.copy(new_phi)
def update_fields_with_RK4(c, dx, dy, dt, M, kappa, boundary_condition):
    Nx, Ny = c.shape
    k1 = np.zeros((Nx, Ny))
    k2 = np.zeros((Nx, Ny))
    k3 = np.zeros((Nx, Ny))
    k4 = np.zeros((Nx, Ny))
    
    # RK4 scheme
    mu = chemical_potential(c, kappa, dx, dy)
    k1 = M * laplacian(mu, dx, dy)
    mu = chemical_potential(c + 0.5 * dt * k1, kappa, dx, dy)
    k2 = M * laplacian(mu, dx, dy)
    mu = chemical_potential(c + 0.5 * dt * k2, kappa, dx, dy)
    k3 = M * laplacian(mu, dx, dy)
    mu = chemical_potential(c + dt * k3, kappa, dx, dy)
    k4 = M * laplacian(mu, dx, dy)
    
    c += dt / 6.0 * (k1 + 2 * k2 + 2 * k3 + k4)
    
    # Boundary conditions
    if boundary_condition == 'Dirichlet':
        c[0, :] = c[-1, :] = c[:, 0] = c[:, -1] = c0
    elif boundary_condition == 'Neumann':
        c[0, :] = c[1, :]
        c[-1, :] = c[-2, :]
        c[:, 0] = c[:, 1]
        c[:, -1] = c[:, -2]
    elif boundary_condition == 'Periodic':
        c[0, :] = c[-1, :]
        c[-1, :] = c[0, :]
        c[:, 0] = c[:, -1]
        c[:, -1] = c[:, 0]
    
    return c

c_hat[:] = fft2(c[0]) #Fourier coefficients

for i in range(1,Nsteps):
    dfdc_hat[:] = fft2(dfdc(c[i-1])) # the FT of the derivative
    dfdc_hat *= dealias # dealising
    c_hat[:] = (c_hat-dt*K2*M*dfdc_hat)/(1+dt*M*kappa*K2**2) # updating in time
    c[i] = ifft2(c_hat).real # inverse fourier transform
for t in range(Nt):
    c = update_fields_with_RK4(c, dx, dy, dt, M, kappa, boundary_condition)